## Notes:
----------

Theory:
We have see a lot of discreapancy with the images, hence for testing puspose we only select images that were last_reviewed after September 2017. We believe that the data atleast reviewed after september should be correct.

1. The accessor photo column had some "nan" values, these were removed.
2. 




## WHAT THE BELOW CELLS DO:

1. We load the metadata file that contains the property metadata for appx 250000 properties including the assessor image.
3. We dont want to experiment on all the images, so we select the ones that were last_reviewed after April 2017. We further filter the images. For land the images that have a improvement level as land are only selected. For House, images with improvement level of anything other than land is taken.
4. We then collect all the assessor images from the links provided in the metadata table. Somtimes the images are not rendered properly, so we filter such images
5. Similarly we gather all the Aerial images using Bing map API call
6. Lastely we acquire all streeside images from Google.
7. We can even gather all the Aerial images using Google map API call

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import logging

logging.basicConfig(level=logging.DEBUG, filename="logfile.log", filemode="w", 
                    format="%(asctime)-15s %(levelname)-8s %(message)s")

from external_data import bing_aerial, google_streetside, google_aerial
from config import pathDict

parent_path = '/Users/sam/All-Program/App-DataSet/HouseClassification/'
meta_data_path = os.path.join(parent_path, "house_metadata_nw.csv")
assessor_image_path = os.path.join(parent_path, "input_images", "assessor_images")
bing_aerial_image_path = os.path.join(parent_path, 'input_images','bing','aerial_images')
google_aerial_image_path = os.path.join(parent_path, 'input_images','google','aerial_images')
streetside_image_path = os.path.join(parent_path, 'input_images','streetside_images')

print('Metadata path: ', meta_data_path)
print('Assessor image path: ', assessor_image_path)
print('Aerial image path (Bing): ', bing_aerial_image_path)
print('Streetside image path: ', streetside_image_path)
print('Aerial image path (Google): ', google_aerial_image_path)

dump_assesor_images = True
rmv_bad_assessor_images = True
dump_aerial_images  = False
rmv_bad_aerial_images = False
dump_streetside_images  = False
rmv_bad_streetside_images = False
dump_aerial_google_images  = True

In [2]:
def collateData(filesPath, fileType="csv"):
    fileDirList = [os.path.join(filesPath, files) for files in
                   os.listdir(filesPath) if files.endswith('.%s' % fileType)]
    if fileType == "csv":
        rowList = []
        # fileDirList = [files for files in listdir if files.endswith('.csv')]
        for files in fileDirList:
            df = pd.read_csv(files, header=0)
            rowList.append(df)
        return pd.concat(rowList)
    elif fileType == "json":
        dataOUT = {}
        for files in fileDirList:
            jsonFilepath = os.path.join(filesPath, files)
            with open(jsonFilepath, 'r') as fileIN:
                dataOUT = merge(dataOUT, json.load(fileIN))
        return dataOUT

# STEP 1: Load Metadata file
------------

In [3]:
metadata = pd.read_csv(meta_data_path)
metadata.columns = ['row_id', 'removed', 'property_id', 'state', 'county_name', 'pin',
       'address_line1', 'address_line2', 'address_city', 'address_zip',
       'zoning', 'improvement_level', 'type', 'exterior',
       'last_reviewed_timestamp', 'gone_timestamp', 'indicator',
       'assessor_photo']
metadata['state'] = metadata['state'].astype('str')
metadata['county_name'] = metadata['county_name'].astype('str')
metadata['pin'] = metadata['pin'].astype('str')
metadata['address_line1'] = metadata['address_line1'].astype('str')
metadata['address_line2'] = metadata['address_line2'].astype('str')
metadata['address_city'] = metadata['address_city'].astype('str')
metadata['address_zip'] = metadata['address_zip'].astype('str')
metadata['zoning'] = metadata['zoning'].astype('str')
metadata['improvement_level'] = metadata['improvement_level'].astype('str')
metadata['type'] = metadata['type'].astype('str')
metadata['exterior'] = metadata['exterior'].astype('str')
metadata['last_reviewed_timestamp'] = pd.to_datetime(metadata['last_reviewed_timestamp'])#.astype('str')
metadata['gone_timestamp'] = pd.to_datetime(metadata['gone_timestamp'])#.astype('str')
metadata['indicator'] = metadata['indicator'].astype('str')
metadata['assessor_photo'] = metadata['assessor_photo'].astype('str')

# Remove the Test Dat from Metadata
metadata = metadata[metadata['removed'] == 0]
print(metadata.shape)
metadata.head(3)

/Users/sam/App-Setup/anaconda/envs/anaconda35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(220364, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
1,1,0,499849,IL,Cook,01-01-100-109-0000,200S HOUGH ST,nan,BARRINGTON,nan,Vacant,Land,nan,nan,NaT,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
2,2,0,499852,IL,Cook,01-01-101-029-0000,201 First,22 # Floor,BARRINGTON,nan,Vacant,Land,4 Unit Apt Bldg,Frame,NaT,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
3,3,0,499854,IL,Cook,01-01-109-001-0000,323 MAIN ST,nan,BARRINGTON,600104305.0,Residential,Improved,2 Flat,Frame,2017-05-25 04:42:26.850,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...


# STEP 2: Create a new Subset
--------------

In [4]:
metadata_lst_rvd_valid = metadata[metadata['last_reviewed_timestamp'] != '']
metadata_lst_rvd_valid['last_reviewed_timestamp'] = pd.to_datetime(metadata_lst_rvd_valid['last_reviewed_timestamp'])
metadata_lst_rvd_valid_2017 = metadata_lst_rvd_valid[((metadata_lst_rvd_valid['last_reviewed_timestamp'].dt.year == 2017) & 
                                                     (metadata_lst_rvd_valid['last_reviewed_timestamp'].dt.month >=1 ) &
#                                                       (metadata_lst_rvd_valid['last_reviewed_timestamp'].dt.month <7) &
                                                      (metadata_lst_rvd_valid['address_city'].str.lower().str.strip().str.match('chicago')) &
                                                      (metadata_lst_rvd_valid['address_line1'] != 'nan') &
                                                      (metadata_lst_rvd_valid['assessor_photo'] != 'nan') &
                                                     (metadata_lst_rvd_valid['indicator'].isin(["Likely House", "Likely Land"]) ))]
image_data = metadata_lst_rvd_valid_2017
print (metadata_lst_rvd_valid_2017.shape)
metadata_lst_rvd_valid_2017.head(2)

(28527, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
6111,6111,0,506508,IL,Cook,09-25-417-038-0000,7334 N OLEANDER AVE,nan,CHICAGO,606314308.0,Residential,Improved,nan,nan,2017-02-15 21:27:08.760,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
8746,8746,0,509513,IL,Cook,10-25-300-045-0000,3125 W HOWARD ST,nan,CHICAGO,606451139.0,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...


### LAND DATA
----------------

In [5]:
land_data = image_data[(image_data['indicator'] == 'Likely Land') & (image_data['improvement_level'] == 'Land')]
print (np.unique(np.array(land_data['improvement_level'])))
print (land_data.shape)
land_data.head(3)

['Land']
(9873, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
8746,8746,0,509513,IL,Cook,10-25-300-045-0000,3125 W HOWARD ST,nan,CHICAGO,606451139.0,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
9339,9339,0,510240,IL,Cook,10-31-417-045-0000,6546 W DEVON AVE,nan,CHICAGO,606312038.0,Vacant,Land,nan,nan,2017-06-22 18:07:04.937,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
9366,9366,0,510267,IL,Cook,10-32-126-007-0000,7046N MCALPIN AVE,nan,CHICAGO,60646.0,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...


### HOUSE DATA
--------------

In [6]:
house_data = image_data[(image_data['indicator'] == 'Likely House') & (image_data['improvement_level'] != 'Land')]
print (np.unique(np.array(house_data['improvement_level'])))
print (house_data.shape)
house_data.head(3)

['Condominium' 'Improved' 'Minor Improvement' 'Mixed Use']
(18392, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
6111,6111,0,506508,IL,Cook,09-25-417-038-0000,7334 N OLEANDER AVE,nan,CHICAGO,606314308.0,Residential,Improved,nan,nan,2017-02-15 21:27:08.760,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
8761,8761,0,509531,IL,Cook,10-25-308-043-0000,3146W FARGO AVE,nan,CHICAGO,60645.0,Residential,Improved,nan,nan,2017-02-10 21:21:30.520,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
8769,8769,0,509539,IL,Cook,10-25-314-003-0000,2949W FARGO AVE,nan,CHICAGO,60645.0,Residential,Improved,nan,nan,2017-02-23 16:25:51.033,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...


### UNKNOWN DATA
-------------

In [7]:
unknown_data = image_data[image_data['indicator'] == 'Unknown']
print (unknown_data.shape)
unknown_data.head(3)

(0, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo


# Step 3: Load Assessor Images
------------

In [8]:
import urllib.request
import requests
import os


def upload_assessor_data(dataIN, dump_path, how_many_images): 
    if how_many_images:
        if how_many_images > len(dataIN):
            raise ValueError ('The random sample cannot exceed the dataIN length')
        random.seed(8792)
        random_indices = random.sample(range(1, len(dataIN)+1), how_many_images)
        random_indices = np.sort(np.array(random_indices, dtype='int32'))    
        dataIN.loc[:,'row_num'] = np.arange(1, len(dataIN)+1)
        dataIN = dataIN[dataIN['row_num'].isin(random_indices)]
    print ('Total images to be loaded is:', dataIN.shape)
    i = 0
    j = 0
    for  row_id, pin, pic_url in np.array(dataIN[['row_id', 'pin', 'assessor_photo']]):
#         print (pic_url)
        response = requests.get(pic_url)
        if response.status_code == 200:
            imagepath = os.path.join(dump_path, '%s.jpg'%(str(pin)))
            with open(imagepath, 'wb') as f:
                f.write(response.content)
            i+=1
        else:
            j += 1
        
        b = "TOTAL IMAGES PARSED ======== %s"
        print (b%(i), end="\r")
    print ('')
    print ("TOTAL FAULTY IMAGES ==== ", j)


#### LOAD LAND IMAGES

In [13]:
if dump_assesor_images:
    assessor_land_dump_path = os.path.join(assessor_image_path, 'land')
    image_already_present = [img.split('.')[0] for img in os.listdir(assessor_land_dump_path)]
    land_data = land_data[~land_data['pin'].isin(image_already_present)]
    upload_assessor_data(dataIN=land_data, dump_path=assessor_land_dump_path, how_many_images=None) # 5000

Total images to be loaded is: (2226, 18)
TOTAL IMAGES PARSED ======== 2225
TOTAL FAULTY IMAGES ====  1


#### LOAD HOUSE IMAGES

In [10]:
if dump_assesor_images:
    assessor_house_dump_path = os.path.join(assessor_image_path, 'house')
    image_already_present = [img.split('.')[0] for img in os.listdir(assessor_house_dump_path)]
    house_data = house_data[~house_data['pin'].isin(image_already_present)]
    upload_assessor_data(dataIN=house_data, dump_path=assessor_house_dump_path, how_many_images=None) # 

Total images to be loaded is: (3822, 18)
TOTAL IMAGES PARSED ======== 3822
TOTAL FAULTY IMAGES ====  0


### REMOVE NOT-AVAILABLE IMAGES
------------------

In [11]:
from scipy import misc
def remove_not_available_data(image_path):
    image_path = [os.path.join(image_path, img) for img in os.listdir(image_path) if img != ".DS_Store"]
    not_available_image_path = '/Users/sam/All-Program/App-DataSet/HouseClassification/input_images/assessor_image_na.jpg'
    bad_data = misc.imread(not_available_image_path)
    bad_data = misc.imresize(bad_data, [160,160])
    faulty_img = 0
    for path in image_path:
        data = misc.imread(path)
        data = misc.imresize(data, [160,160])
        if (bad_data == data).all():
            faulty_img += 1
            os.remove(path)
    print ('Total Faulty images: ', faulty_img)

In [12]:
if rmv_bad_assessor_images:
    assessor_land_dump_path = os.path.join(assessor_image_path, 'land')
    assessor_house_dump_path = os.path.join(assessor_image_path, 'house')
    remove_not_available_data(image_path=assessor_land_dump_path)
    remove_not_available_data(image_path=assessor_house_dump_path)

Total Faulty images:  292
Total Faulty images:  1380


# Step 4: Load Aerial Images:
------------

### Aerial Land and House DUMP

In [7]:
land_house_data = pd.concat([land_data, house_data])
print (land_house_data.shape)
pd.concat([land_house_data.head(2), land_house_data.tail(2)])

(20854, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
34,34,0,499887,IL,Cook,01-01-406-017-0000,901 PRAIRIE AVE,nan,BARRINGTON,nan,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
35,35,0,499888,IL,Cook,01-01-407-008-0000,920 PRAIRIE AVE,nan,BARRINGTON,nan,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
223392,223392,0,1426092,IL,Cook,17-30-105-004-0000,2051 W CERMAK,nan,CHICAGO,60608,Residential,Mixed Use,nan,nan,2017-06-23 15:07:43.277,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
223393,223393,0,1426093,IL,Cook,17-30-112-038-0000,2337 W 23RD,nan,CHICAGO,60608,Residential,Improved,nan,nan,2017-06-23 15:07:43.277,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...


In [8]:
if dump_aerial_images:
    bing_search.get_image_address(land_house_data, batch_size=500, get_stats=True)

[['01-01-406-017-0000' '901 PRAIRIE AVE ' 'BARRINGTON' 'nan' 'Likely Land']
 ['01-01-407-008-0000' '920 PRAIRIE AVE' 'BARRINGTON' 'nan' 'Likely Land']
 ['01-12-103-004-0000' '0 GROVE AVE ' 'BARRINGTON' 'nan' 'Likely Land']
 ['01-28-101-002-0000' '161 LINDEN AVE' 'DUNDEE' '60010.0' 'Likely Land']
 ['02-09-108-005-0000' '1223 W DUNDEE RD ' 'PALATINE' '600749212.0'
  'Likely Land']]


#### Load all the Aerial statistics data and check if the url is Nan

In [26]:
bing_aerial_stats_data = collateData(filesPath=pathDict['bing_aerial_stats_path'], fileType="csv")
bing_aerial_stats_data['img_url'] = bing_aerial_stats_data['img_url'].astype('str')
nan_data = bing_aerial_stats_data[bing_aerial_stats_data['img_url'] == 'nan']
nan_data.head()
#### There are only two images, but this were never stores into the image file, because of the exception handled.

/Users/sam/All-Program/App-DataSet/HouseClassification/statistics/aerial_images/data_loader


,pin,city,zip,address,lat,lon,zoom_lvl,image_fetch,indicator,img_url
226,08-30-201-007-0000,SCHAUMBURG,NaN,0 RTE #1-BOX 313A,NaN,NaN,NaN,NaN,Likely Land,nan
51,32-14-200-016-0000,LANSING,NaN,0 ROUTE 394 & JOE RD,NaN,NaN,NaN,NaN,Likely Land,nan


# Step 5: Load Streetside Images:
------------

In [8]:
land_house_data = pd.concat([land_data, house_data])
print (land_house_data.shape)
pd.concat([land_house_data.head(2), land_house_data.tail(2)])

(26851, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
34,34,0,499887,IL,Cook,01-01-406-017-0000,901 PRAIRIE AVE,nan,BARRINGTON,nan,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
35,35,0,499888,IL,Cook,01-01-407-008-0000,920 PRAIRIE AVE,nan,BARRINGTON,nan,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
259193,259193,0,1480786,IL,Lake,17-31-102-026,434 LAKESIDE MANOR RD,nan,HIGHLAND PARK,60035,Residential,Improved,nan,nan,2017-11-01 15:10:56.890,NaT,Likely House,http://apps01.lakecountyil.gov/assessor/images...
259194,259194,0,1480787,IL,Lake,17-31-302-063,315 SHERIDAN RD,nan,HIGHLAND PARK,60035,Residential,Improved,nan,nan,2017-11-01 15:10:56.890,NaT,Likely House,http://apps01.lakecountyil.gov/assessor/images...


In [11]:
streetside_land_dump_path = os.path.join(streetside_image_path, 'land')
streetside_house_dump_path = os.path.join(streetside_image_path, 'house')

print (streetside_land_dump_path)
print (streetside_house_dump_path)

/Users/sam/All-Program/App-DataSet/HouseClassification/input_images/streetside_images/land
/Users/sam/All-Program/App-DataSet/HouseClassification/input_images/streetside_images/house


In [10]:
if dump_streetside_images:
     google_street_map.get_image_address(land_house_data, batch_size=500, get_stats=True)

### Remove the not-available images

In [33]:
from scipy import misc
def remove_not_available_data(image_path):
    image_path = [os.path.join(image_path, img) for img in os.listdir(image_path) if img != ".DS_Store"]
    not_available_image_path = '/Users/sam/All-Program/App-DataSet/HouseClassification/input_images/streetside_image_na.jpg'
    bad_data = misc.imread(not_available_image_path)
    bad_data = misc.imresize(bad_data, [160,160])
    faulty_img = 0
    for num, path in enumerate(image_path):
        try:
            data = misc.imread(path)
#             print(data.shape)
            data = misc.imresize(data, [160,160])
            if data.shape[2] != 3:                     #  Somtimes the image comes with 4 channels
                faulty_img += 1
                b = "TOTAL Faulty Images Found ======== %s"
                print (b%(faulty_img), end="\r")
                os.remove(path)
            elif (bad_data == data).all():
                faulty_img += 1
                b = "TOTAL Faulty Images Found ======== %s"
                print (b%(faulty_img), end="\r")
                os.remove(path)
            else:
                pass
        except OSError:
            print ('Opps cant read the file: %s',os.path.basename(path))
            
if rmv_bad_streetside_images:
#     remove_not_available_data(streetside_land_dump_path)
    remove_not_available_data(streetside_house_dump_path)

# Step 6: Load Aerial Images (Google Static Maps):
------------

### Gather Data Already Processed: (Due to exceed in daily quota google blocks web calls. Hence here we collate data that we have already gathered before to avoid redundancy)

In [8]:
google_aerial_stats_path = pathDict['google_aerial_stats_path']
google_aerial_stats = collateData(filesPath=google_aerial_stats_path, fileType="csv")
print (google_aerial_stats.shape)
google_aerial_stats.head()

(30510, 9)


,pin,address,city,lat,lon,loc_type,indicator,meta_url,img_url
0,20-28-324-003-0000,7809 S LOWE AVE,CHICAGO,41.7522,-87.6401,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
1,20-20-411-019-0000,6847 S CARPENTER ST,CHICAGO,41.7692,-87.6504,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
2,17-19-324-019-0000,2115 W 21ST PL,CHICAGO,41.8528,-87.679,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
3,14-31-111-038-1001,2319 W LYNDALE ST,CHICAGO,41.9224,-87.6859,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
4,20-11-305-017-1001,5115 S UNIVERSITY AVE,CHICAGO,41.8018,-87.5978,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...


In [9]:
processed_images = google_aerial_stats.dropna()
processed_images = processed_images[processed_images['meta_url'] != 'EXCEED']
print (processed_images.shape)
processed_images.head()

(30364, 9)


,pin,address,city,lat,lon,loc_type,indicator,meta_url,img_url
0,20-28-324-003-0000,7809 S LOWE AVE,CHICAGO,41.7522,-87.6401,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
1,20-20-411-019-0000,6847 S CARPENTER ST,CHICAGO,41.7692,-87.6504,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
2,17-19-324-019-0000,2115 W 21ST PL,CHICAGO,41.8528,-87.679,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
3,14-31-111-038-1001,2319 W LYNDALE ST,CHICAGO,41.9224,-87.6859,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
4,20-11-305-017-1001,5115 S UNIVERSITY AVE,CHICAGO,41.8018,-87.5978,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...


### Empty the Directory and store only the processed data in batches:

In [10]:
import shutil
SURE = input("Sure? Delete all files from  %s "%(str(google_aerial_stats_path)))
shutil.rmtree(google_aerial_stats_path + '/')
os.makedirs(google_aerial_stats_path)

Sure? Delete all files from  /Users/sam/All-Program/App-DataSet/HouseClassification/statistics/aerial_images/google yes


In [11]:
processed_images = processed_images[['pin', 'address', 'city', 'lat', 'lon','loc_type', 'indicator', 'meta_url', 'img_url']]
processed_images.to_csv(os.path.join(google_aerial_stats_path, '%s_%s.csv'%(str(0), str(len(processed_images)-1))), index=None)

### Aerial Land and House DUMP (Google)

In [12]:
land_house_data = pd.concat([land_data, house_data])
print (land_house_data.shape)
pd.concat([land_house_data.head(2), land_house_data.tail(2)])

(28265, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
8746,8746,0,509513,IL,Cook,10-25-300-045-0000,3125 W HOWARD ST,nan,CHICAGO,606451139.0,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
9339,9339,0,510240,IL,Cook,10-31-417-045-0000,6546 W DEVON AVE,nan,CHICAGO,606312038.0,Vacant,Land,nan,nan,2017-06-22 18:07:04.937,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
223392,223392,0,1426092,IL,Cook,17-30-105-004-0000,2051 W CERMAK,nan,CHICAGO,60608,Residential,Mixed Use,nan,nan,2017-06-23 15:07:43.277,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
223393,223393,0,1426093,IL,Cook,17-30-112-038-0000,2337 W 23RD,nan,CHICAGO,60608,Residential,Improved,nan,nan,2017-06-23 15:07:43.277,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...


### Remove from the land_house_data where the PIN is already processed

In [13]:
land_house_data = land_house_data[~land_house_data['pin'].isin(np.array(processed_images['pin'], dtype=str))]
print (land_house_data.shape)
land_house_data.head()

(8704, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
13945,13945,0,515364,IL,Cook,12-24-219-042-0000,3714 N OSCEOLA AVE,nan,CHICAGO,606343306.0,Residential,Land,nan,nan,2017-02-10 22:04:53.280,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
16124,16124,0,517888,IL,Cook,13-03-116-011-0000,6107N FOREST GLEN AVE,nan,CHICAGO,60646.0,Residential,Land,nan,nan,2017-02-10 22:04:53.280,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
16557,16557,0,518409,IL,Cook,13-07-132-022-0000,6905W FARRAGUT AVE,nan,CHICAGO,60656.0,Vacant,Land,nan,nan,2017-02-10 22:04:53.280,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
17190,17190,0,519165,IL,Cook,13-11-304-024-0000,5101N MONTICELLO AVE,nan,CHICAGO,60625.0,Residential,Land,nan,nan,2017-02-10 22:04:53.280,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
18086,18086,0,520161,IL,Cook,13-13-401-017-0000,2585 W MONTROSE AVE,nan,CHICAGO,606181505.0,Residential,Land,nan,nan,2017-02-10 22:04:53.280,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...


#### Shuffle

In [14]:
from sklearn.utils import shuffle
land_house_data = shuffle(land_house_data)
print (land_house_data.shape)
land_house_data.head()

(8704, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
187858,187858,0,922795,IL,Cook,14-21-106-034-1078,663 W GRACE ST,516,CHICAGO,60613.0,Residential,Condominium,Condo Unit,Brick,2017-02-10 21:21:30.520,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
174094,174094,0,757589,IL,Cook,25-03-311-044-0000,9217 S PRAIRIE AVE,nan,CHICAGO,60619.0,Residential,Improved,Row House,Brick,2017-02-10 21:21:30.520,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
215721,215721,0,1210177,IL,Cook,20-10-300-054-0000,5156 S WABASH AVE,nan,CHICAGO,606153311,Residential,Improved,nan,nan,2017-06-09 22:51:31.103,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
96388,96388,0,608300,IL,Cook,20-32-429-021-0000,860 W 86TH PL,nan,CHICAGO,606203218.0,Residential,Improved,nan,nan,2017-02-21 21:40:11.010,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
153986,153986,0,671924,IL,Cook,32-20-418-011-0000,1608 ABERDEEN ST,nan,CHICAGO HEIGHTS,60411.0,Residential,Improved,2 Flat,Frame,2017-03-03 01:21:23.187,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...


In [ ]:
if dump_aerial_google_images:
    google_aerial.fetch_google_aerial_images(land_house_data, batch_size=100, get_stats=True)

# Step 7: Here we randomly select a subset of the images, the conditions for selection are:

* Fisrst we run "get_valid_land_house_ids" under the main.py module to get images that were properly fetched for all different types of property images i.e "assessor", 'streetside', 'bing_aerial'.
* Now that we have Google Aerial Images metadata, we perform the below operation to do some supposedly "bad data" removal
    * A image should have a valid address_line1 and city. The address should not start with '0 0 GROVE AVE', '0 SUMMIT ST', '0 VACANT PROPERTY'
    * The location_type should not be "RANGE_INTERPOLATED"
    * The Lat and Lon should have a 4 or more than 4 decimal precision.
    

In [12]:
from main import get_valid_land_house_ids

valid_land_pins, valid_house_pins = get_valid_land_house_ids(images_per_label=None)
print ('Total qualifies valid pins each Label: ', len(valid_land_pins), len(valid_house_pins))

Total qualifies valid pins each Label:  6670 6670


In [13]:
google_stats_data = collateData(pathDict['google_aerial_stats_path'])
print (google_stats_data.shape)
google_stats_data.head()

(30364, 9)


,pin,address,city,lat,lon,loc_type,indicator,meta_url,img_url
0,20-28-324-003-0000,7809 S LOWE AVE,CHICAGO,41.752219,-87.640134,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
1,20-20-411-019-0000,6847 S CARPENTER ST,CHICAGO,41.769213,-87.650386,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
2,17-19-324-019-0000,2115 W 21ST PL,CHICAGO,41.852836,-87.679003,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
3,14-31-111-038-1001,2319 W LYNDALE ST,CHICAGO,41.922418,-87.685920,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...
4,20-11-305-017-1001,5115 S UNIVERSITY AVE,CHICAGO,41.801807,-87.597783,ROOFTOP,Likely House,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/staticmap...


# Step 8: Overlay the building bounding box on Google satellite images:

In [17]:
overlay_bbox = False
if overlay_bbox:
    google_stats_data = utl.collateData(pathDict['google_aerial_stats_path'])
    print('Shape: Google Stats images ', google_stats_data.shape)

    data_to_model = google_stats_data[(google_stats_data['loc_type'] != 'RANGE_INTERPOLATED') & (
    google_stats_data['city'].str.lower().str.strip().str.match('chicago')) & (
                                      ~google_stats_data['address'].str.lower().str.strip().str.match('0'))]
    print('Shape: Images for Overlaying: ', data_to_model.shape)
    overlay_parcel_on_images(data_to_model)

# Step 9: Select a subset of images for train and test